In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()
# Access environment variables
api_key = os.getenv('API_KEY')
api_type = os.getenv('OPENAI_API_TYPE')
model_name = os.getenv('MODEL_NAME')

base_url = os.getenv('BASE_URL')

#DM_db_url = os.getenv('DRUGMATRIX_DB_URL')
#database_url = os.getenv('DATABASE_URL')
debug_mode = os.getenv('DEBUG')
engin = os.getenv('MODEL_ENGINE')
version = os.getenv('OPENAI_API_VERSION')


os.environ["OPENAI_API_TYPE"] = api_type
os.environ["OPENAI_API_BASE"] = "https://litellm.toxpipe.niehs.nih.gov"
os.environ["OPENAI_API_KEY"] =  api_key
os.environ["AZURE_OPENAI_ENDPOINT"] = base_url
os.environ["OPENAI_API_VERSION"] = version 

print(f"version : {api_type}")
print(f"base url : {base_url}")
print(f"API_KEY : {api_key}")
print(f"engin: {engin}")
print(f"model_name: {model_name}")



version : azure
base url : https://litellm.toxpipe.niehs.nih.gov
API_KEY : sk-dSTROKbqfoD8L2WDNDhwXA
engin: azure-gpt-4o
model_name: azure-gpt-4


In [2]:
#from llama_index.multi_modal_llms.azure_openai import AzureOpenAIMultiModal
from llama_index.legacy.multi_modal_llms.azure_openai import AzureOpenAIMultiModal


In [3]:
import base64
import requests
from llama_index.core.schema import ImageDocument


In [4]:
azure_openai_mm_llm = AzureOpenAIMultiModal(
    model="gpt-4-vision-preview",#"gpt-4o",
    engine=engin,#"azure-gpt-4o",  # Replace with your deployment name
    azure_endpoint=base_url,#azure_endpoint,  # Replace with your endpoint URL
    api_key=api_key,  # Replace with your API key
    api_version=version #"2023-05-15"
)

In [5]:

#file_path =  "./jupyter_apps/LLM/imgs/ZF_LDTT.png" 
file_path =  "./imgs/ZF_LDTT.png" 
# Open the image
with open(file_path, "rb") as image_file:
        image_data = image_file.read()
base64str_ZF = base64.b64encode(image_data).decode("utf-8")

image_document_ZF = ImageDocument(image=base64str_ZF, image_mimetype="image/png")

#from PIL import Image        
#image_data = Image.open(file_path)
# Display the image (optional)
#image.show() 

In [6]:
 complete_response_ZF = azure_openai_mm_llm.complete(
    prompt="Describe the images as an alternative text",
    image_documents=[image_document_ZF],
)

In [7]:
print(complete_response_ZF)

The image is a database schema diagram created using dbdiagram.io. It consists of multiple tables with their respective fields and relationships. Here is a detailed description of each table and its fields:

1. **sources**
   - source_name (varchar)
   - source_code (varchar)

2. **chemicals**
   - chemical_id (integer)
   - chemical_name (varchar)
   - chemical_code (varchar)
   - casrn (varchar)
   - preferred_name (varchar)

3. **plates**
   - plate_id (integer)
   - plate_name (varchar)
   - plate_type (varchar)
   - plate_screen_time (timestamp)
   - plate_stage (varchar)

4. **well_types**
   - well_type_id (integer)
   - well_description (varchar)

5. **devices_data**
   - devices_data_id (integer)
   - endpoint_id (integer)
   - well_type (varchar)
   - well_position (varchar)
   - concentration_unit (varchar)
   - vehicle (varchar)
   - solvent (varchar)
   - cell_density (varchar)
   - stim_bladder_deflated (boolean)

6. **behavior_data**
   - devices_data_id (integer)
   - e

In [21]:
def describe (question):
    complete_response_ZF = azure_openai_mm_llm.complete(
    #prompt="Describe the images as an alternative text",
    prompt=question,
    image_documents=[image_document_ZF],
)
    return complete_response_ZF
    

In [22]:
import gradio as gr 
def greet(name): return "Hello " + name + "!" 
def prodiction (question): 
    answer = describe (question)
    return answer
demo = gr.Interface(fn=prodiction, inputs="textbox", outputs="textbox") 
if __name__ == "__main__": 
    demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://6152bf3f578dfa787e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
import gradio as gr
from llama_index.legacy.multi_modal_llms.azure_openai import AzureOpenAIMultiModal
import llama_index
#import ollama

def format_history(msg: str, history: list[list[str, str]], system_prompt: str):
    chat_history = [{"role": "system", "content":system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})  
    chat_history.append({"role": "user", "content": msg})
    return chat_history

def init_loadimages():
     #file_path =  "./jupyter_apps/LLM/imgs/ZF_LDTT.png" 
    file_path =  "./imgs/ZF_LDTT.png" 
    
    try:
        # Read the uploaded image file
        with open(file_path, "rb") as img_file:
            image_data = img_file.read()
        
        # Convert image to base64 and create ImageDocument
        base64str = base64.b64encode(image_data).decode("utf-8")
        image_document = ImageDocument(image=base64str, image_mimetype="image/png")
        return image_document
         
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
def generate_response(msg: str, history: list[list[str, str]], system_prompt: str):
    chat_history = format_history(msg, history, system_prompt)
    image_document = init_loadimages()
    response = azure_openai_mm_llm.complete(
        prompt="Describe the images as an alternative text",
        image_documents=[image_document],
    )
      # Check response type and print it
    print("Response type:", type(response))
    #print("Response :", response)
    
    #from llama_index.core import Response
    #text = chat_response.message.content
    #response = Response(text)
    
    # Handle CompletionResponse
    if isinstance(response, llama_index.legacy.core.llms.types.CompletionResponse):
        try:
            content = response.text  # Assuming the content is in the "text" attribute
            #content = complete_response_ZF.text
            #content = data.get('content')
            #print("content type : ",type(content))
            print("content : ",content)
            max_length = 200  # Adjust this value as needed
            shortened_content = content[:max_length] + "..."
            #yield [("assistant", shortened_content)]             
            yield [{"role": "system", "content":shortened_content}]
            
            #table_descriptions = shortened_content.split("\n\n")  # Split by double newline
            #for description in table_descriptions:
                #yield [("description",description)]
            #yield [("assistant",  "my test content")]
        except AttributeError:
            # Handle potential absence of "text" attribute
            yield {"assistant",  "Error: Unexpected response format 1"}
        
    else:
        # Handle other unexpected formats (if needed)
        yield {"assistant",  "Error: Unexpected response format 2"}
    
    
     
    #message = response
    #print (response)
    #yield message
#gr.Markdown ("## kskssksksk")
chatbot = gr.ChatInterface(
                
                generate_response,
                chatbot=gr.Chatbot(
                        avatar_images=["./imgs/user.png", "./imgs/Chatbot.jpg"],
                        height="64vh"
                    ),
                additional_inputs=[
                    gr.Textbox(
                        "Behave as if you are professional datascientist.",
                        label="System Prompt"
                    )
                ],
                
    
                title="🦜🔗 LLama Chatbot database images",
                description="Feel free to ask any question about the zebra file database.",
                theme="soft",
                submit_btn="⬅ Send",
                #retry_btn="🔄 Regenerate Response",
                #undo_btn="↩ Delete Previous",
                #clear_btn="🗑️ Clear Chat"
)
chatbot.launch( share=True)
#chatbot.launch(server_name='https://jupyter-gpu.niehs.nih.gov/', server_port=7860)

/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://21676d4c39f6958bdc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Response type: <class 'llama_index.legacy.core.llms.types.CompletionResponse'>
content :  The image is a database schema diagram created using dbdiagram.io. It consists of several tables with their respective columns and relationships. Here is a description of each table and its columns:

1. **sources**
   - source_name (varchar)
   - source_id (integer, primary key)

2. **chemicals**
   - chemical_id (integer, primary key)
   - chemical_code (varchar)
   - casrn (varchar)
   - preferred_name (varchar)

3. **plates**
   - plate_id (integer, primary key)
   - plate_name (varchar)
   - plate_type (varchar)
   - plate_screen_time (timestamp)
   - plate_stage (varchar)

4. **well_types**
   - well_type_id (integer, primary key)
   - well_description (varchar)

5. **devices_data**
   - devices_data_id (integer, primary key)
   - source_id (integer, foreign key)
   - well_type (varchar)
   - well_position (varchar)
   - concentration_unit (varchar)
   - vehicle (varchar)
   - solvent (varcha

In [45]:
#print(response.text) 
print(complete_response_ZF.text)

The image is a database schema diagram created using dbdiagram.io. It consists of several tables with their respective columns and relationships. The tables and their columns are as follows:

1. **sources**
   - source_name (varchar)
   - source_id (integer)

2. **chemicals**
   - chemical_id (integer)
   - chemical_name (varchar)
   - chemical_code (varchar)
   - casrn (varchar)
   - preferred_name (varchar)

3. **plates**
   - plate_id (integer)
   - plate_name (varchar)
   - plate_type (varchar)
   - plate_screen_time (integer)
   - plate_screen_stage (varchar)

4. **well_types**
   - well_type_id (integer)
   - well_description (varchar)

5. **devices_data**
   - devices_data_id (integer)
   - source_id (integer)
   - device_id (integer)
   - well_type (varchar)
   - well_position (varchar)
   - concentration_unit (varchar)
   - vehicle (varchar)
   - solvent (varchar)
   - cell_line (varchar)
   - cell_density (varchar)
   - stim_bladder_deflated (boolean)

6. **bins**
   - bin_id

In [22]:
import gradio as gr
#import ollama

def format_history(msg: str, history: list[list[str, str]], system_prompt: str):
    chat_history = [{"role": "system", "content":system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})  
    chat_history.append({"role": "user", "content": msg})
    return chat_history

def init_loadimages():
     #file_path =  "./jupyter_apps/LLM/imgs/ZF_LDTT.png" 
    file_path =  "./imgs/ZF_LDTT.png" 
    
    try:
        # Read the uploaded image file
        with open(file_path, "rb") as img_file:
            image_data = img_file.read()
        
        # Convert image to base64 and create ImageDocument
        base64str = base64.b64encode(image_data).decode("utf-8")
        image_document = ImageDocument(image=base64str, image_mimetype="image/png")
        return image_document
         
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
def generate_response(msg: str, history: list[list[str, str]], system_prompt: str):
    chat_history = format_history(msg, history, system_prompt)
    image_document = init_loadimages()
    response = azure_openai_mm_llm.complete(
        prompt="Describe the images as an alternative text",
        image_documents=[image_document],
    )
      # Check response type and print it
    print("Response type:", type(response))
    
    # Handle CompletionResponse
    if isinstance(response, llama_index.legacy.core.llms.types.CompletionResponse):
        try:
            content = response.text  # Assuming the content is in the "text" attribute
            content = """{ 'role': 'assistant',
    'content': 'The image is a database schema diagram with several tables, including sources, chemicals, plates, well_types}""" 
            content ="""{'chat_message':'hello'}"""
            
            content = {'role': 'assistant','content': 'HHHHHHHHello, how can I assist you today?'}
            

            yield {"role": "assistant", "content": content}
        except AttributeError:
            # Handle potential absence of "text" attribute
            yield {"role": "assistant", "content": "Error: Unexpected response format"}
    else:
        # Handle other unexpected formats (if needed)
        yield {"role": "assistant", "content": "Error: Unexpected response format"}
    
    
    # Check if response is a string (assuming it's the desired format)
    if isinstance(response, str):
        # Yield the complete response as a single message
        yield {"role": "assistant", "content": response}
        
    else:
        if isinstance(response, list):
          # Handle list format (assuming first element is content)
            try:
                content = response[0]  # Assuming content is at index 0 (adjust if needed)
                content = {'role': 'assistant','content': 'Hello, how can I assist you today?'}
                if content:
                    yield {"role": "assistant", "content": content}
            except IndexError:
                # Handle potential issues with accessing list elements
                yield {"role": "assistant", "content": "Error: Unexpected response format"}
        else:
            # Handle other unexpected formats (if needed)
            yield {"role": "assistant", "content": "Error: Unexpected response format 1 "}
        # Handle other response formats (if necessary)
        yield {"role": "assistant", "content": "Error: Unexpected response format 2 "}
     
    #message = response
    #print (response)
    #yield message

chatbot = gr.ChatInterface(
                generate_response,
                chatbot=gr.Chatbot(
                        avatar_images=["./imgs/user.png", "./imgs/Chatbot.jpeg"],
                        height="64vh"
                    ),
                additional_inputs=[
                    gr.Textbox(
                        "Behave as if you are professional writer.",
                        label="System Prompt"
                    )
                ],
                title="LLama-2 (7B) Chatbot using 'Ollama'",
                description="Feel free to ask any question.",
                theme="soft",
                submit_btn="⬅ Send",
                #retry_btn="🔄 Regenerate Response",
                #undo_btn="↩ Delete Previous",
                #clear_btn="🗑️ Clear Chat"
)
chatbot.launch( share=True)
#chatbot.launch(server_name='https://jupyter-gpu.niehs.nih.gov/', server_port=7860)

/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7880
* Running on public URL: https://93f04755c61d4af684.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Chatbot for the sql images

In [11]:
import gradio as gr
#import ollama

def format_history(msg: str, history: list[list[str, str]], system_prompt: str):
    chat_history = [{"role": "system", "content":system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})  
    chat_history.append({"role": "user", "content": msg})
    return chat_history

def init_loadimages():
     #file_path =  "./jupyter_apps/LLM/imgs/ZF_LDTT.png" 
    file_path =  "./imgs/ZF_LDTT.png" 
    
    try:
        # Read the uploaded image file
        with open(file_path, "rb") as img_file:
            image_data = img_file.read()
        
        # Convert image to base64 and create ImageDocument
        base64str = base64.b64encode(image_data).decode("utf-8")
        image_document = ImageDocument(image=base64str, image_mimetype="image/png")
        return image_document
         
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
def generate_response(msg: str, history: list[list[str, str]], system_prompt: str):
    chat_history = format_history(msg, history, system_prompt)
    image_document = init_loadimages()
    response = azure_openai_mm_llm.complete(
        prompt="Describe the images as an alternative text",
        image_documents=[image_document],
    )
    message = response
    #print (response)
    for partial_resp in response:
        try:
            # Check if "message" is a dictionary and access content
            if isinstance(partial_resp["message"], dict):
                token = partial_resp["message"]["content"]
            # Handle list format (example)
            elif isinstance(partial_resp["message"], list):
                for message_item in partial_resp["message"]:
                    content = message_item.get("content", None)
                    if content:
                        message += content
                        break  # Stop iterating after finding content (optional)
            elif isinstance(partial_resp["message", tuple]):
                content = partial_resp["message"][0]
                if content:
                  message += content
            else:
                # Handle other formats if needed
                pass
            message += token
        except KeyError:
            # Handle potential absence of "message" key
            message += "Error: Could not access response content."
        yield message

chatbot = gr.ChatInterface(
                generate_response,
                chatbot=gr.Chatbot(
                        avatar_images=["./imgs/user.png", "./imgs/Chatbot.jpg"],
                        height="64vh"
                    ),
                additional_inputs=[
                    gr.Textbox(
                        "Behave as if you are professional writer.",
                        label="System Prompt"
                    )
                ],
                title="LLama-2 (7B) Chatbot using 'Ollama'",
                description="Feel free to ask any question.",
                theme="soft",
                submit_btn="⬅ Send",
                #retry_btn="🔄 Regenerate Response",
                #undo_btn="↩ Delete Previous",
                #clear_btn="🗑️ Clear Chat"
)
chatbot.launch( share=True)
#chatbot.launch(server_name='https://jupyter-gpu.niehs.nih.gov/', server_port=7860)

/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://4889a6d9b344eb88df.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/blocks.py", line 2013, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/gradio/blocks.py", line 1578, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packag